In [93]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl 

In [94]:
# Create fuzzy variables

# Inputs
Temperature = ctrl.Antecedent(np.arange(0, 101, 1), 'Temperature(F)')
Wind_Chill = ctrl.Antecedent(np.arange(0, 101, 1), 'Wind_Chill(F)')
Humidity = ctrl.Antecedent(np.arange(0, 101, 1), 'Humidity(%)')
Pressure = ctrl.Antecedent(np.arange(0, 101, 1), 'Pressure(in)')
Wind_Speed = ctrl.Antecedent(np.arange(0, 101, 1), 'Wind_Speed(mph)')

# Outputs
Support = ctrl.Consequent(np.arange(0, 1.01, 0.01), 'Support')
#Severity = ctrl.Consequent(np.arange(0, 5, 1), 'Severity')



In [95]:
# Define membership functions


# Put the dynamic thresholds here to cast fuzy variables


# Input Fuzzy Variables:

Temperature['minimal'] = fuzz.trimf(Temperature.universe, [0, 0, 46])
Temperature['low'] = fuzz.trimf(Temperature.universe, [0, 46, 58])
Temperature['moderate'] = fuzz.trimf(Temperature.universe, [46, 58, 69])
Temperature['high'] = fuzz.trimf(Temperature.universe, [58, 69, 78.1])
Temperature['maximum'] = fuzz.trimf(Temperature.universe, [69, 78.1, 78.1])

Wind_Chill['minimal'] = fuzz.trimf(Wind_Chill.universe, [0, 0, 45])
Wind_Chill['low'] = fuzz.trimf(Wind_Chill.universe, [0, 45, 60])
Wind_Chill['moderate'] = fuzz.trimf(Wind_Chill.universe, [45, 60, 64])
Wind_Chill['high'] = fuzz.trimf(Wind_Chill.universe, [60, 64, 76])
Wind_Chill['maximum'] = fuzz.trimf(Wind_Chill.universe, [64, 76, 76])

Humidity['minimal'] = fuzz.trimf(Humidity.universe, [0, 0, 44])
Humidity['low'] = fuzz.trimf(Humidity.universe, [0, 44, 60])
Humidity['moderate'] = fuzz.trimf(Humidity.universe, [44, 60, 73])
Humidity['high'] = fuzz.trimf(Humidity.universe, [60, 73, 87])
Humidity['maximum'] = fuzz.trimf(Humidity.universe, [73, 87, 87])

Pressure['minimal'] = fuzz.trimf(Pressure.universe, [0, 0, 29.16])
Pressure['low'] = fuzz.trimf(Pressure.universe, [0, 29.16, 29.69])
Pressure['moderate'] = fuzz.trimf(Pressure.universe, [29.16, 29.69, 29.91])
Pressure['high'] = fuzz.trimf(Pressure.universe, [29.69, 29.91, 30.04])
Pressure['maximum'] = fuzz.trimf(Pressure.universe, [29.91, 30.04, 30.04])

Wind_Speed['minimal'] = fuzz.trimf(Wind_Speed.universe, [0, 0, 3])
Wind_Speed['low'] = fuzz.trimf(Wind_Speed.universe, [0, 3, 6])
Wind_Speed['moderate'] = fuzz.trimf(Wind_Speed.universe, [3, 6, 8])
Wind_Speed['high'] = fuzz.trimf(Wind_Speed.universe, [6, 8, 12])
Wind_Speed['maximum'] = fuzz.trimf(Wind_Speed.universe, [8, 12, 12])

Support['rare'] = fuzz.trimf(Support.universe, [0, 0, 0.20])
Support['infrequent'] = fuzz.trimf(Support.universe, [0, 0.20, 0.40])
Support['occasional'] = fuzz.trimf(Support.universe, [0.20, 0.40, 0.60])
Support['frequent'] = fuzz.trimf(Support.universe, [0.40, 0.60, 0.80])
Support['very_frequent'] = fuzz.trimf(Support.universe, [0.60, 0.80, 0.80])


In [96]:
# Define fuzzy rules

rule1 = ctrl.Rule( Pressure['low'] | Humidity['low'], Support['very_frequent'] )
rule2 = ctrl.Rule( Pressure['low'] & Humidity['low'], Support['frequent'])
rule3 = ctrl.Rule( Temperature['maximum'] | Wind_Chill['maximum'], Support['occasional'])
rule4 = ctrl.Rule( Temperature['maximum'] & Humidity['low'], Support['occasional'])
rule5 = ctrl.Rule( Wind_Chill['maximum'], Support['occasional'])
rule6 = ctrl.Rule( Wind_Speed['moderate'], Support['infrequent'])
rule7 = ctrl.Rule( Wind_Chill['low'] & Humidity['minimal'], Support['infrequent'])
rule8 = ctrl.Rule( Pressure['minimal'] & Wind_Speed['low'], Support['infrequent'])
rule9 = ctrl.Rule( Wind_Chill['low'] & Temperature['moderate'], Support['rare'])
rule10 = ctrl.Rule( Humidity['moderate'] | Pressure['high'], Support['rare'])


In [97]:
rules = [
         rule1, rule2, rule3, 
         rule4, rule5, rule6, 
         rule7, rule8, rule9,
         rule10
        ]

In [98]:
# Create control system

system = ctrl.ControlSystem(rules)
simulation = ctrl.ControlSystemSimulation(system)

In [99]:
type(69.06)

float

In [75]:
# Set input values 

simulation.input['Temperature(F)'] = 69.06
simulation.input['Humidity(%)'] = 50

simulation.input['Wind_Chill(F)'] = 50
simulation.input['Pressure(in)'] = 50
simulation.input['Wind_Speed(mph)'] = 5


In [76]:
# Compute fuzzy logic

simulation.compute()


In [77]:
# Get output value

Support = simulation.output['Support']

In [78]:
print('Detected Support:', Support)

Detected Support: 0.36146532509338447


In [79]:
import requests
from datetime import datetime, timedelta

def get_weather(api_key, lat, lon, timestamp):
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = f"{base_url}lat={lat}&lon={lon}&appid={api_key}&units=imperial"
    
    response = requests.get(complete_url)
    data = response.json()

    if data["cod"] != "404":
        main_data = data["main"]
        visibility = data["visibility"] / 1609.34  # Convert meters to miles
        wind_data = data["wind"]
        
        temperature_F = main_data["temp"]
        humidity_percent = main_data["humidity"]
        pressure_inch = main_data["pressure"] / 33.863886666667  # Convert hPa to inches of mercury
        visibility_mi = visibility
        wind_speed_mph = wind_data["speed"]
        
        # Additional features
        wind_chill_F = data["main"]["feels_like"]

        weather_info = {
            "Time": timestamp,
            "Temperature(F)": temperature_F,
            "Wind_Chill(F)": wind_chill_F,
            "Humidity(%)": humidity_percent,
            "Pressure(in)": pressure_inch,
            "Wind_Speed(mph)": wind_speed_mph
        }
        
        return weather_info
    else:
        print("City not found or API error")
        return None


In [80]:
api_key = "ce060a4759ed58f5151e42ae95b8df59"
latitude = 37.77
longitude = 122.41

# Calculate timestamp for tomorrow at 12 o'clock midday
tomorrow_midday = datetime.now() + timedelta(days=1)
tomorrow_midday = tomorrow_midday.replace(hour=12, minute=0, second=0, microsecond=0)
timestamp_trip = tomorrow_midday.strftime("%Y-%m-%d %H:%M:%S")
timestamp_today = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


get_weather(api_key, latitude, longitude, timestamp_today)

{'Time': '2023-08-27 14:14:29',
 'Temperature(F)': 69.37,
 'Wind_Chill(F)': 70.09,
 'Humidity(%)': 87,
 'Pressure(in)': 29.825282902159195,
 'Wind_Speed(mph)': 8.72}

In [89]:
import requests
from datetime import datetime, timedelta

def get_weather_forecast(api_key, lat, lon, start_time, end_time):
    base_url = "http://api.openweathermap.org/data/2.5/forecast?"
    complete_url = f"{base_url}lat={lat}&lon={lon}&appid={api_key}&units=imperial"
    
    response = requests.get(complete_url)
    data = response.json()

    if data["cod"] == "200":
        forecast_list = data["list"]
        weather_info_list = []
        
        for forecast in forecast_list:
            forecast_time = datetime.utcfromtimestamp(forecast["dt"])
            
            # Check if the forecast time is within the desired range
            if start_time <= forecast_time <= end_time:
                main_data = forecast["main"]
                wind_data = forecast["wind"]
                
                temperature_F = main_data["temp"]
                humidity_percent = main_data["humidity"]
                pressure_inch = main_data["pressure"] / 33.863886666667  # Convert hPa to inches of mercury
                wind_speed_mph = wind_data["speed"]
                wind_chill_F = forecast["main"]["feels_like"]
                
                weather_info = {
                    "Time": forecast_time.strftime("%Y-%m-%d %H:%M:%S"),
                    "Temperature(F)": temperature_F,
                    "Wind_Chill(F)": wind_chill_F,
                    "Humidity(%)": humidity_percent,
                    "Pressure(in)": pressure_inch,
                    "Wind_Speed(mph)": wind_speed_mph
                }
                
                weather_info_list.append(weather_info)
        
        return weather_info_list
    else:
        print("API error")
        return []

if __name__ == "__main__":
    api_key = "ce060a4759ed58f5151e42ae95b8df59"
    latitude = 37.77
    longitude = 122.41
    
    # Calculate the time range (10 hours before and 10 hours after the input time)
    input_time = datetime.now().replace(hour=12, minute=0, second=0, microsecond=0)
    start_time = input_time - timedelta(hours=10)
    end_time = input_time + timedelta(hours=10)
    
    weather_data_list = get_weather_forecast(api_key, latitude, longitude, start_time, end_time)
    
    if weather_data_list:
        print("Weather Forecast for the Specified Time Range:")
        forecast_values = []
        for weather_data in weather_data_list:
            forecast_values.append(weather_data)
            for key, value in weather_data.items():
                print(f"{key}: {value}")
            print()


Weather Forecast for the Specified Time Range:
Time: 2023-08-27 12:00:00
Temperature(F): 69.06
Wind_Chill(F): 69.62
Humidity(%): 84
Pressure(in): 29.825282902159195
Wind_Speed(mph): 9.75

Time: 2023-08-27 15:00:00
Temperature(F): 68.63
Wind_Chill(F): 69.24
Humidity(%): 86
Pressure(in): 29.825282902159195
Wind_Speed(mph): 11.74

Time: 2023-08-27 18:00:00
Temperature(F): 68.58
Wind_Chill(F): 69.17
Humidity(%): 86
Pressure(in): 29.79575291908775
Wind_Speed(mph): 10.29

Time: 2023-08-27 21:00:00
Temperature(F): 70.07
Wind_Chill(F): 70.95
Humidity(%): 89
Pressure(in): 29.73669295294486
Wind_Speed(mph): 19.95



In [90]:
forecast_values

[{'Time': '2023-08-27 12:00:00',
  'Temperature(F)': 69.06,
  'Wind_Chill(F)': 69.62,
  'Humidity(%)': 84,
  'Pressure(in)': 29.825282902159195,
  'Wind_Speed(mph)': 9.75},
 {'Time': '2023-08-27 15:00:00',
  'Temperature(F)': 68.63,
  'Wind_Chill(F)': 69.24,
  'Humidity(%)': 86,
  'Pressure(in)': 29.825282902159195,
  'Wind_Speed(mph)': 11.74},
 {'Time': '2023-08-27 18:00:00',
  'Temperature(F)': 68.58,
  'Wind_Chill(F)': 69.17,
  'Humidity(%)': 86,
  'Pressure(in)': 29.79575291908775,
  'Wind_Speed(mph)': 10.29},
 {'Time': '2023-08-27 21:00:00',
  'Temperature(F)': 70.07,
  'Wind_Chill(F)': 70.95,
  'Humidity(%)': 89,
  'Pressure(in)': 29.73669295294486,
  'Wind_Speed(mph)': 19.95}]

In [114]:
if __name__ == "__main__":
    api_key = "ce060a4759ed58f5151e42ae95b8df59"
    # San Francisco Location, Cluster 11
    latitude = 37.77
    longitude = 122.41
    
    # Calculate the time range (10 hours before and 10 hours after the input time)
    input_time = datetime.now().replace(hour=12, minute=0, second=0, microsecond=0)
    start_time = input_time - timedelta(hours=10)
    end_time = input_time + timedelta(hours=10)
    
    weather_data_list = get_weather_forecast(api_key, latitude, longitude, start_time, end_time)
    
    if weather_data_list:
        print("Weather Forecast for the Specified Time Range:")
        for weather_data in weather_data_list:
            for key, value in weather_data.items():
                print(f"{key}: {value}")
            print()

        system = ctrl.ControlSystem(rules)
        simulation = ctrl.ControlSystemSimulation(system)
        
        min_support = float('inf')  # Initialize with a large value
        min_support_time = None
            
        for forecast_data in weather_data_list:
                  
            simulation.input['Temperature(F)'] = forecast_data['Temperature(F)']
            simulation.input['Humidity(%)'] = forecast_data['Humidity(%)']
            simulation.input['Wind_Chill(F)'] = forecast_data['Wind_Chill(F)']
            simulation.input['Pressure(in)'] = forecast_data['Pressure(in)']
            simulation.input['Wind_Speed(mph)'] = forecast_data['Wind_Speed(mph)']

            # Compute fuzzy logic
            simulation.compute()

            # Get and store output value
            detected_support = simulation.output['Support']
            supports.append(detected_support)

            # Check for minimum support
            if detected_support < min_support:
                min_support = detected_support
                min_support_time = forecast_data['Time']

Weather Forecast for the Specified Time Range:
Time: 2023-08-27 12:00:00
Temperature(F): 69.06
Wind_Chill(F): 69.62
Humidity(%): 84
Pressure(in): 29.825282902159195
Wind_Speed(mph): 9.75

Time: 2023-08-27 15:00:00
Temperature(F): 68.63
Wind_Chill(F): 69.24
Humidity(%): 86
Pressure(in): 29.825282902159195
Wind_Speed(mph): 11.74

Time: 2023-08-27 18:00:00
Temperature(F): 68.58
Wind_Chill(F): 69.17
Humidity(%): 86
Pressure(in): 29.79575291908775
Wind_Speed(mph): 10.29

Time: 2023-08-27 21:00:00
Temperature(F): 70.07
Wind_Chill(F): 70.95
Humidity(%): 89
Pressure(in): 29.73669295294486
Wind_Speed(mph): 19.95



In [119]:
if __name__ == "__main__":
    api_key = "ce060a4759ed58f5151e42ae95b8df59"
    # San Francisco Location, Cluster 11
    latitude = 37.77
    longitude = 122.41
    
    # Calculate the time range (10 hours before and 10 hours after the input time)
    input_time = datetime.now().replace(hour=12, minute=0, second=0, microsecond=0)
    start_time = input_time - timedelta(hours=10)
    end_time = input_time + timedelta(hours=10)
    
    weather_data_list = get_weather_forecast(api_key, latitude, longitude, start_time, end_time)
    
    if weather_data_list:
        print("Weather Forecast for the Specified Time Range:")
        for weather_data in weather_data_list:
            for key, value in weather_data.items():
                print(f"{key}: {value}")
            print()

        system = ctrl.ControlSystem(rules)
        simulation = ctrl.ControlSystemSimulation(system)
            
        min_support = float('inf')  # Initialize with a large value
        min_support_time = None

        for forecast_data in weather_data_list:
                  
            simulation.input['Temperature(F)'] = forecast_data['Temperature(F)']
            simulation.input['Humidity(%)'] = forecast_data['Humidity(%)']
            simulation.input['Wind_Chill(F)'] = forecast_data['Wind_Chill(F)']
            simulation.input['Pressure(in)'] = forecast_data['Pressure(in)']
            simulation.input['Wind_Speed(mph)'] = forecast_data['Wind_Speed(mph)']

            # Compute fuzzy logic
            simulation.compute()

            # Get and store output value
            detected_support = simulation.output['Support']
            supports.append(detected_support)

            # Check for minimum support
            if detected_support < min_support:
                min_support = detected_support
                min_support_time = forecast_data['Time']

            print(f"Time: {forecast_data['Time']}, Support: {detected_support}")

        print("List of Supports:", supports)
        print(f"Time related to min support: {min_support_time}")


Weather Forecast for the Specified Time Range:
Time: 2023-08-27 12:00:00
Temperature(F): 69.06
Wind_Chill(F): 69.62
Humidity(%): 84
Pressure(in): 29.825282902159195
Wind_Speed(mph): 9.75

Time: 2023-08-27 15:00:00
Temperature(F): 68.63
Wind_Chill(F): 69.24
Humidity(%): 86
Pressure(in): 29.825282902159195
Wind_Speed(mph): 11.74

Time: 2023-08-27 18:00:00
Temperature(F): 68.58
Wind_Chill(F): 69.17
Humidity(%): 86
Pressure(in): 29.79575291908775
Wind_Speed(mph): 10.29

Time: 2023-08-27 21:00:00
Temperature(F): 70.07
Wind_Chill(F): 70.95
Humidity(%): 89
Pressure(in): 29.73669295294486
Wind_Speed(mph): 19.95

Time: 2023-08-27 12:00:00, Support: 0.3845851756559618
Time: 2023-08-27 15:00:00, Support: 0.3840860744939569
Time: 2023-08-27 18:00:00, Support: 0.3937681716409405
Time: 2023-08-27 21:00:00, Support: 0.4104750117393314
List of Supports: [0.36146532509338447, 0.36146532509338447, 0.36146532509338447, 0.36146532509338447, 0.3845851756559618, 0.3840860744939569, 0.3937681716409405, 0.410

In [121]:
print(f"Suggested Time: {min_support_time}")

Suggested Time: 2023-08-27 15:00:00


In [122]:
# Fuzzy Time Wise Recommender